## Imputer:
- Substitui valores ausentes
- Usa uma estratégia de substituição: média, mediana ou moda
- Pode ainda substituir qualquer outro valor (por exemplo, zero) usando parâmetro setMissingValue

Valor default 


In [2]:
from pyspark.sql import SparkSession

from pyspark.ml.feature import Imputer

spark = (
    SparkSession
    .builder
    .appName('OneHotEncoding')
    .getOrCreate()
)
spark

In [3]:
df_cars = (
    spark.read.format('csv')
    .option('inferSchema', True)
    .option('delimiter', ';')
    .option('header', True)
    .load('data/CarrosNAN.csv')
)

df_cars.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|       null|             39|2875| null|        0|          1|      4|          4|110|
|    228|        0|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        0|       null|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        0|        360|            315|null| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
df_cars.printSchema()
df_cars.count()


root
 |-- Consumo: integer (nullable = true)
 |-- Cilindros: integer (nullable = true)
 |-- Cilindradas: integer (nullable = true)
 |-- RelEixoTraseiro: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Tempo: integer (nullable = true)
 |-- TipoMotor: integer (nullable = true)
 |-- Transmissao: integer (nullable = true)
 |-- Marchas: integer (nullable = true)
 |-- Carburadors: integer (nullable = true)
 |-- HP: integer (nullable = true)



32

In [7]:
_df_cars_input = Imputer(
    inputCols=['Cilindradas', 'Peso'],
    outputCols=['Cilindradas_cleaned', 'Peso_cleaned'],
).fit(df_cars).transform(df_cars)

_df_cars_input.select('Cilindradas', 'Peso', 'Cilindradas_cleaned', 'Peso_cleaned').show(5)

+-----------+----+-------------------+------------+
|Cilindradas|Peso|Cilindradas_cleaned|Peso_cleaned|
+-----------+----+-------------------+------------+
|        160| 262|                160|         262|
|       null|2875|                848|        2875|
|        108| 232|                108|         232|
|       null|3215|                848|        3215|
|        360|null|                360|        1318|
+-----------+----+-------------------+------------+
only showing top 5 rows



In [10]:
_df_cars_input = Imputer(
    inputCol='Cilindros',
    outputCol='Cilindros_cleaned',
    strategy='median',
    missingValue=0
).fit(df_cars).transform(df_cars).select('Cilindros', 'Cilindros_cleaned').show()

+---------+-----------------+
|Cilindros|Cilindros_cleaned|
+---------+-----------------+
|        6|                6|
|        6|                6|
|        0|                6|
|        0|                6|
|        0|                6|
|        6|                6|
|        8|                8|
|        4|                4|
|        4|                4|
|        0|                6|
|        6|                6|
|        8|                8|
|        8|                8|
|        8|                8|
|        8|                8|
|        8|                8|
|        8|                8|
|        4|                4|
|        4|                4|
|        4|                4|
+---------+-----------------+
only showing top 20 rows

